In [2]:
from src.utils.database import get_async_engine, get_async_session, get_async_engine_session
from src.sechema.inference import Video, Frame, Inference

In [3]:
from sqlalchemy import create_engine
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey, TIMESTAMP, func, ARRAY, text

In [4]:
user = "postgres"
password = "postgrespassword"
host = "localhost"
port = "5432"
database = "ai_db"
engine, AsyncSessionLocal = get_async_engine_session(user, password, host, port, database)

In [10]:
schema_name = "video_inference"
Base = declarative_base()

# Ensure schema exists
async with engine.begin() as conn:
    await conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name}"))
    # Create all tables
    await conn.run_sync(Base.metadata.create_all)

2026-02-18 21:28:46,530 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2026-02-18 21:28:46,531 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 21:28:46,538 INFO sqlalchemy.engine.Engine select current_schema()
2026-02-18 21:28:46,539 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 21:28:46,544 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2026-02-18 21:28:46,544 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 21:28:46,549 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:28:46,550 INFO sqlalchemy.engine.Engine CREATE SCHEMA IF NOT EXISTS video_inference
2026-02-18 21:28:46,551 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()
2026-02-18 21:28:46,557 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
# Insert example data
async with AsyncSessionLocal() as session:
    video = Video(file_path="../Data/Videos/leak.mp4")
    frame = Frame(frame_number=1, frame_path="frames/sample_frame_1.jpg")
    video.frames.append(frame)

    session.add(video)
    await session.commit()       # must await in async
    await session.refresh(video) # refresh to get ID
    await session.refresh(frame) # optional, get frame ID

    print("Video saved with ID:", video.id)
    print("Frame saved with ID:", frame.id)

2026-02-18 21:28:49,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:28:49,188 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.videos (file_name, file_path) VALUES ($1::VARCHAR, $2::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at
2026-02-18 21:28:49,190 INFO sqlalchemy.engine.Engine [generated in 0.00124s] (None, '../Data/Videos/leak.mp4')
2026-02-18 21:28:49,196 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.NotNullViolationError'>: null value in column "file_name" of relation "videos" violates not-null constraint
DETAIL:  Failing row contains (4, null, ../Data/Videos/leak.mp4, 2026-02-18 15:58:49.192218).
[SQL: INSERT INTO video_inference.videos (file_name, file_path) VALUES ($1::VARCHAR, $2::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at]
[parameters: (None, '../Data/Videos/leak.mp4')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

#### Insert the videos first and them frames

In [12]:
async def insert_video(video_path, video_name):
    async with AsyncSessionLocal() as session:
        try:
            video = Video(file_path=video_path, file_name=video_name)

            session.add(video)
            await session.commit()
            await session.refresh(video)

            print("Video saved with ID:", video.id)

        except Exception as e:
            await session.rollback()
            print("Error:", e)

# In Jupyter Notebook:
await insert_video('Data/Videos/PeopleWalkCCTV.mp4', 'PeopleWalkCCTV')

2026-02-18 21:28:54,092 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:28:54,094 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.videos (file_name, file_path) VALUES ($1::VARCHAR, $2::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at
2026-02-18 21:28:54,095 INFO sqlalchemy.engine.Engine [cached since 4.907s ago] ('PeopleWalkCCTV', 'Data/Videos/PeopleWalkCCTV.mp4')
2026-02-18 21:28:54,101 INFO sqlalchemy.engine.Engine COMMIT
2026-02-18 21:28:54,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:28:54,129 INFO sqlalchemy.engine.Engine SELECT video_inference.videos.id, video_inference.videos.file_name, video_inference.videos.file_path, video_inference.videos.created_at 
FROM video_inference.videos 
WHERE video_inference.videos.id = $1::INTEGER
2026-02-18 21:28:54,130 INFO sqlalchemy.engine.Engine [generated in 0.00109s] (5,)
Video saved with ID: 5
2026-02-18 21:28:54,137 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# Insert video first

async def insert_frame():
    async with AsyncSessionLocal() as session:
        try:
            # 1. Create Video object
            frame = Frame(frame_number=1, frame_path="frames/sample_frame_2.jpg", video_id=1)
            
            # Alternatively, you can still use the relationship:
            # video.frames.append(frame)

            # 4. Add frame to session and commit
            session.add(frame)
            await session.commit()
            await session.refresh(frame)

            print("Frame saved with ID:", frame.id)

            print("Video saved with ID:", video.id)
        except Exception as e:
            print(e)

2026-02-18 21:02:16,327 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:02:16,328 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.frames (video_id, frame_number, frame_path) VALUES ($1::INTEGER, $2::INTEGER, $3::VARCHAR) RETURNING video_inference.frames.id, video_inference.frames.created_at
2026-02-18 21:02:16,330 INFO sqlalchemy.engine.Engine [cached since 6.029s ago] (1, 5, 'frames/sample_frame_2.jpg')
2026-02-18 21:02:16,333 INFO sqlalchemy.engine.Engine COMMIT
2026-02-18 21:02:16,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 21:02:16,339 INFO sqlalchemy.engine.Engine SELECT video_inference.frames.id, video_inference.frames.video_id, video_inference.frames.frame_number, video_inference.frames.frame_path, video_inference.frames.created_at 
FROM video_inference.frames 
WHERE video_inference.frames.id = $1::INTEGER
2026-02-18 21:02:16,340 INFO sqlalchemy.engine.Engine [cached since 5.997s ago] (3,)
Frame saved with ID: 3
Video saved with ID: None


### Insert-Infernce Data

In [ ]:
# Suppose YOLO detected 2 objects in this frame
detections_data = [
    {
        "class_name": "person",
        "confidence": 0.92,
        "x_min": 100,
        "y_min": 150,
        "x_max": 300,
        "y_max": 450
    },
    {
        "class_name": "car",
        "confidence": 0.88,
        "x_min": 400,
        "y_min": 200,
        "x_max": 700,
        "y_max": 500
    }
]

for det in detections_data:
    detection = Inference(frame_id=frame.id, **det)
    session.add(detection)

session.commit()
session.close()

print("Video, frames, and YOLO detections stored successfully!")

#### Test With src files

In [8]:
from src.utils.database import get_async_engine, get_async_session, get_async_engine_session
user = "postgres"
password = "postgrespassword"
host = "localhost"
port = "5432"
database = "ai_db"
engine, AsyncSessionLocal = get_async_engine_session(user, password, host, port, database)

In [11]:
from src.database.inference import create_video, create_frame

In [10]:
async with AsyncSessionLocal() as session:
    video = await create_video(session, "PeopleWalkCCTV", "../Data/Videos/PeopleWalkCCTV.mp4")
    print(video.id)

2026-02-19 02:08:50,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-19 02:08:50,506 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.videos (file_name, file_path) VALUES ($1::VARCHAR, $2::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at
2026-02-19 02:08:50,508 INFO sqlalchemy.engine.Engine [cached since 135.8s ago] ('PeopleWalkCCTV', '../Data/Videos/PeopleWalkCCTV.mp4')
2026-02-19 02:08:50,512 INFO sqlalchemy.engine.Engine COMMIT
2026-02-19 02:08:50,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-19 02:08:50,517 INFO sqlalchemy.engine.Engine SELECT video_inference.videos.id, video_inference.videos.file_name, video_inference.videos.file_path, video_inference.videos.created_at 
FROM video_inference.videos 
WHERE video_inference.videos.id = $1::INTEGER
2026-02-19 02:08:50,518 INFO sqlalchemy.engine.Engine [cached since 135.8s ago] (7,)
7
2026-02-19 02:08:50,521 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
import cv2
async with AsyncSessionLocal() as session:
    cap = cv2.VideoCapture('Data/Videos/PeopleWalkCCTV.mp4')
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        await create_frame(session, 7, count , f"frame{count}")

2026-02-19 02:12:47,870 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-19 02:12:47,873 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.frames (video_id, frame_number, frame_path) VALUES ($1::INTEGER, $2::INTEGER, $3::VARCHAR) RETURNING video_inference.frames.id, video_inference.frames.created_at
2026-02-19 02:12:47,874 INFO sqlalchemy.engine.Engine [generated in 0.00105s] (7, 1, 'frame1')


2026-02-19 02:12:47,880 INFO sqlalchemy.engine.Engine COMMIT
2026-02-19 02:12:47,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-19 02:12:47,886 INFO sqlalchemy.engine.Engine SELECT video_inference.frames.id, video_inference.frames.video_id, video_inference.frames.frame_number, video_inference.frames.frame_path, video_inference.frames.created_at 
FROM video_inference.frames 
WHERE video_inference.frames.id = $1::INTEGER
2026-02-19 02:12:47,886 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (4,)
2026-02-19 02:12:47,893 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.frames (video_id, frame_number, frame_path) VALUES ($1::INTEGER, $2::INTEGER, $3::VARCHAR) RETURNING video_inference.frames.id, video_inference.frames.created_at
2026-02-19 02:12:47,893 INFO sqlalchemy.engine.Engine [cached since 0.02083s ago] (7, 2, 'frame2')
2026-02-19 02:12:47,896 INFO sqlalchemy.engine.Engine COMMIT
2026-02-19 02:12:47,899 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02